In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import regex as re
import csv

In [2]:
# Load the CSV file into a DataFrame
file_path = '../Data/all_stats.csv'
stats_df = pd.read_csv(file_path)
print(f'{file_path} uploaded')
stats_df.tail(5)

../Data/all_stats.csv uploaded


,player_name,game_id,kills,deaths,assists,adr,fk,fd,clutches,aces
475,Cryocells,48,25.0,13.0,1.0,199.0,7.0,2.0,0,0
476,eeiu,48,19.0,9.0,5.0,161.0,2.0,1.0,0,0
477,Asuna,48,14.0,13.0,24.0,132.0,2.0,1.0,1,0
478,bang,48,14.0,12.0,12.0,137.0,1.0,1.0,0,0
479,Boostio,48,12.0,15.0,6.0,138.0,1.0,2.0,0,0


In [3]:
# Load the CSV file into a DataFrame
file_path = '../Data/scores_data.csv'
games_df = pd.read_csv(file_path)
print(f'{file_path} uploaded')
games_df.tail(5)

../Data/scores_data.csv uploaded


,game_id,map_name,home_team,away_team,map_duration,home_score,away_score
58,58,TBD,LOUD,100T,-,0,0
59,59,TBD,LOUD,100T,-,0,0
60,60,TBD,SEN,G2,-,0,0
61,61,TBD,SEN,G2,-,0,0
62,62,TBD,SEN,G2,-,0,0


In [4]:
# Drop rows where both home_score and away_score are 0
games_df = games_df[~((games_df['home_score'] == 0) & (games_df['away_score'] == 0))]

In [5]:
games_df['game_id'] = games_df['game_id'] + 1

In [6]:
games_df.tail()

,game_id,map_name,home_team,away_team,map_duration,home_score,away_score
43,44,Icebox,NRG,C9,49:20,10,13
44,45,Sunset,NRG,C9,59:18,12,14
45,46,Lotus,EG,100T,1:06:50,10,13
46,47,Bind,EG,100T,55:39,13,10
47,48,Ascent,EG,100T,46:01,7,13


In [7]:
# If series_id doesn't exist, create it as a placeholder
if 'series_id' not in games_df:
    games_df['series_id'] = range(1, len(games_df['home_team']) + 1)

In [8]:
# Group by home_team and away_team, and create a new DataFrame
series_df = games_df.groupby(['home_team', 'away_team']).agg({
    'series_id': 'first'
}).reset_index()

# # Drop the total scores as they are no longer needed
games_df = games_df.drop(columns=['series_id'])

In [9]:
series_df

,home_team,away_team,series_id
0,EG,100T,46
1,EG,FUR,13
2,EG,LEV,26
3,FUR,100T,5
4,G2,C9,10
5,G2,KRÜ,36
6,LEV,100T,20
7,LOUD,EG,7
8,LOUD,FUR,28
9,LOUD,LEV,39


In [10]:
# Sort series_df by series_id
series_df = series_df.sort_values(by='series_id')

In [11]:
# Correct series_id to be sequential
series_df['series_id'] = range(1, len(series_df) + 1)

In [12]:
# Reset index
series_df = series_df.reset_index(drop=True)

In [13]:
series_df

,home_team,away_team,series_id
0,MIBR,LEV,1
1,SEN,NRG,2
2,FUR,100T,3
3,LOUD,EG,4
4,G2,C9,5
5,EG,FUR,6
6,SEN,KRÜ,7
7,LOUD,MIBR,8
8,LEV,100T,9
9,NRG,G2,10


In [14]:
# Initialize a new column in games_df to store series_id
games_df['series_id'] = None

# Iterate through games_df and assign series_id based on matching teams in series_df
for index, row in games_df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Find matching series_id in series_df
    matching_series = series_df[(series_df['home_team'] == home_team) & (series_df['away_team'] == away_team)]
    
    if not matching_series.empty:
        series_id = matching_series.iloc[0]['series_id']
        games_df.at[index, 'series_id'] = series_id

In [15]:
games_df

,game_id,map_name,home_team,away_team,map_duration,home_score,away_score,series_id
0,1,Ascent,MIBR,LEV,1:05:24,9,13,1
1,2,Icebox,MIBR,LEV,45:04,7,13,1
2,3,Lotus,SEN,NRG,59:21,13,8,2
3,4,Sunset,SEN,NRG,1:00:54,14,12,2
4,5,Icebox,FUR,100T,1:12:51,14,12,3
5,6,Haven,FUR,100T,1:00:50,16,14,3
6,7,Lotus,LOUD,EG,52:39,13,7,4
7,8,Sunset,LOUD,EG,40:31,6,13,4
8,9,Icebox,LOUD,EG,55:13,11,13,4
9,10,Ascent,G2,C9,1:05:05,13,11,5


In [16]:
# Reorder columns
games_df = games_df[['series_id', 'game_id', 'map_name', 'home_team', 'away_team', 'map_duration', 'home_score', 'away_score']]

In [17]:
# Calculate round differences for each game
games_df['home_round_difference'] = games_df['home_score'] - games_df['away_score']
games_df['away_round_difference'] = games_df['away_score'] - games_df['home_score']

C:\Users\brendanm\AppData\Local\Temp\ipykernel_12044\1065950809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df['home_round_difference'] = games_df['home_score'] - games_df['away_score']
C:\Users\brendanm\AppData\Local\Temp\ipykernel_12044\1065950809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df['away_round_difference'] = games_df['away_score'] - games_df['home_score']


In [18]:
games_df.head()

,series_id,game_id,map_name,home_team,away_team,map_duration,home_score,away_score,home_round_difference,away_round_difference
0,1,1,Ascent,MIBR,LEV,1:05:24,9,13,-4,4
1,1,2,Icebox,MIBR,LEV,45:04,7,13,-6,6
2,2,3,Lotus,SEN,NRG,59:21,13,8,5,-5
3,2,4,Sunset,SEN,NRG,1:00:54,14,12,2,-2
4,3,5,Icebox,FUR,100T,1:12:51,14,12,2,-2


In [19]:
# Initialize empty lists to store total round differences and total maps
home_round_diff_totals = []
away_round_diff_totals = []
total_maps = []

# Loop through each series_id in series_df
for series_id in series_df['series_id']:
    # Filter games_df for the current series_id
    subset = games_df[games_df['series_id'] == series_id]
    
    # Calculate total round differences for home and away teams
    home_round_diff_total = subset['home_round_difference'].sum()
    away_round_diff_total = subset['away_round_difference'].sum()
    
    # Calculate total maps for the current series_id
    total_maps_count = len(subset)
    
    # Append totals to the lists
    home_round_diff_totals.append(home_round_diff_total)
    away_round_diff_totals.append(away_round_diff_total)
    total_maps.append(total_maps_count)

In [20]:
series_df['home_round_difference'] = home_round_diff_totals
series_df['away_round_difference'] = away_round_diff_totals
series_df['total_maps'] = total_maps

In [21]:
# Reorder columns
series_df = series_df[['series_id', 'home_team', 'away_team','home_round_difference', 'away_round_difference', 'total_maps']]
series_df

,series_id,home_team,away_team,home_round_difference,away_round_difference,total_maps
0,1,MIBR,LEV,-10,10,2
1,2,SEN,NRG,7,-7,2
2,3,FUR,100T,4,-4,2
3,4,LOUD,EG,-3,3,3
4,5,G2,C9,-1,1,3
5,6,EG,FUR,17,-17,2
6,7,SEN,KRÜ,5,-5,2
7,8,LOUD,MIBR,8,-8,3
8,9,LEV,100T,12,-12,2
9,10,NRG,G2,-11,11,2


In [22]:
# Save DataFrame to CSV file
series_df.to_csv('../Data/series_df.csv', index=False)

In [23]:
# # Drop the total scores as they are no longer needed
games_df = games_df.drop(columns=['home_round_difference', 'away_round_difference'])

In [24]:
games_df.to_csv('../Data/games_data_updated.csv', index=False)